# Spanish Electricity Pool Forecasting: Exploratory Data Analysis

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import skew, kurtosis, pearsonr,  linregress as linreg

from sklearn.preprocessing import MinMaxScaler,StandardScaler


In [ ]:
import config
sns.set(
    rc=config.set_plot_features(), style="darkgrid"
       )

# 1. Input and prepare data

In [ ]:
PATH = './data/'
infile1 = 'quand_daily.csv'
infile2 = 'esios_daily.csv'
outfile = 'data_spot_price_forecasting.csv'

## 1.1. Load financial data

In [ ]:
fin_data = pd.read_csv(PATH + infile1, parse_dates = ["date"], index_col=["date"])
fin_data.info()

## 1.2. Load ESIOS data

In [ ]:
esios_data = pd.read_csv(PATH + infile2, parse_dates = ["date"], index_col=["date"])
esios_data.info()

## 1.3 Convert ESIOS format
Pivot column "indicator" to convert a long dataset to a wide one, also filter geo

In [ ]:
esios_data.groupby(["indicador", "geo_id"]).size()

In [ ]:
esios_data[(esios_data['indicador'] =='Precio mercado SPOT Diario')].groupby(["geo_id", "geo_name"]).size()

In [ ]:
esios_data = esios_data[esios_data["geo_id"].isin([8741, 3])]
esios_data[(esios_data['indicador'] =='Precio mercado SPOT Diario')].groupby(["geo_id", "geo_name"]).size()

In [ ]:
esios_rename_dict = {'Precio mercado SPOT Diario': 'spot_price',
    'Generación programada P48 Exportación Total': 'GP48_export',
    'Generación programada P48 Importación Total': 'GP48_import',
    'Generación programada P48 Cogeneración': 'GP48_cogen',
    'Generación programada P48 Eólica': 'GP48_wind',
    'Generación programada P48 Carbón': 'GP48_coal',
    'Generación programada P48 Consumo bombeo': 'GP48_pumping_con',
    'Generación programada P48 Gas Natural Cogeneración': 'GP48_cogen_ng',
    'Generación programada P48 Biomasa': 'GP48_biomass',
    'Generación programada P48 Solar térmica': 'GP48_thermosolar',
    'Generación programada P48 Solar fotovoltaica': 'GP48_PV',
    'Generación programada P48 Ciclo combinado': 'GP48_CC',
    'Generación programada P48 Turbinación bombeo': 'GP48_pumping_tur',
    'Generación programada P48 Nuclear': 'GP48_nuclear',
    'Generación programada P48 Hidráulica no UGH': 'GP48_hydro_noUGH',
    'Generación programada P48 Hidráulica UGH': 'GP48_hydro_UGH',
    'Generación programada P48 UGH + no UGH': 'GP48_hydro_total',
    'Generación programada PBF total': 'GP48_total',
    'Demanda real': 'Demand_actual',
    'Demanda programada': 'Demad_schedulled',
    'Demanda prevista': 'Demand_forecasted'}

In [ ]:
esios_data_wide = esios_data.pivot(columns='indicador', values='value').rename(columns=esios_rename_dict)
esios_data_wide['spot_price'] = esios_data_wide['spot_price']
esios_data_wide.head()

## 1.4 Merge data and set daily index

In [ ]:
esios_select = ['spot_price', 'GP48_total', 'GP48_export', 'GP48_import', 'GP48_cogen', 'GP48_wind', 'GP48_coal', 
                'GP48_biomass', 'GP48_thermosolar', 'GP48_PV', 'GP48_CC','GP48_nuclear',
                'GP48_pumping_con', 'GP48_pumping_tur', 'GP48_hydro_total']
data = esios_data_wide[esios_select].merge(fin_data, how='left', left_index=True, right_index=True)\
    .asfreq("D")
data[['GP48_pumping_con', 'GP48_pumping_tur','GP48_export', 'GP48_import']] =\
    data[['GP48_pumping_con', 'GP48_pumping_tur','GP48_export', 'GP48_import']].fillna(0.0)
data.info()

In [ ]:
data.head()

In [ ]:
data.isna().sum()

In [ ]:
data.tail()

In [ ]:
data[["spot_price", "GP48_total"]].describe().T

In [ ]:
data.corr()

In [ ]:
data.head()

# 2. Basic Feature Engineering 

## 2.1 Time features

In [ ]:
data["year"] = data.index.year.astype(int)
data["qtr"] = data.index.quarter.astype(int)
data["mon"] = data.index.month.astype(int)
data["week"] = data.index.week.astype(int)
data["weekday"] = data.index.weekday.astype(int)

data["qtr_idx"] = data["year"]*10+data["qtr"]
data["mon_idx"] = data["year"]*100+data["mon"]
data["flg_isWeekend"] = np.where(data["weekday"].isin([5,6]),1,0)

data[["year","qtr","mon","week","weekday","flg_isWeekend"]].head(10)

In [ ]:
features_time = ["week_sin","week_cos","weekday_sin","weekday_cos"]

data["week_sin"] = np.sin((data['week']-1)*(2*np.pi/52))
data["week_cos"] = np.cos((data['week']-1)*(2*np.pi/52))

data["weekday_sin"] = np.sin((data['weekday'])*(2*np.pi/7))
data["weekday_cos"] = np.cos((data['weekday'])*(2*np.pi/7))

data[["week_sin", "week_cos"]].plot(figsize=(8,6))
plt.title("Cyclical features")
plt.show()

## 2.2. Derived features

### 2.2.1. Compute generation share by techology

In [ ]:
pct_compute =['GP48_export', 'GP48_import',
 'GP48_cogen','GP48_wind', 'GP48_coal',
 'GP48_biomass', 'GP48_thermosolar', 'GP48_PV',
 'GP48_CC', 'GP48_nuclear',
 'GP48_pumping_con','GP48_pumping_tur', 'GP48_hydro_total']

features_gen_pct = [x + "_pct" for x in pct_compute]

data[features_gen_pct] = data[pct_compute].divide(data["GP48_total"], axis=0)

data[features_gen_pct].head()

### 2.2.2. Compute price quantiles by quater and a weekend indicator

In [ ]:
def apply_quantiles(x, q25, q50, q75):
    if x < q25:
        return 0
    elif x<q50:
        return 1
    elif x<q75:
        return 2
    else:
        return 3

In [ ]:
data_q_spot = data[["spot_price", "year", "qtr", "flg_isWeekend"]].copy() 
q_cols = []

for q in [0.25, 0.5, 0.75]:
    q_grp = data.groupby(["year", "qtr", "flg_isWeekend"])["spot_price"].quantile(q).to_frame("q_" + str(q*100))
    data_q_spot = data_q_spot.merge(q_grp, "left", left_on=["year", "qtr", "flg_isWeekend"], right_index=True)
    q_cols.append("q_" + str(q*100))

data_q_spot["spot_price_q"] = data_q_spot[["spot_price"] + q_cols].apply(lambda x: apply_quantiles(x[0], x[1], x[2], x[3]), axis=1)
data_q_spot.tail(10)

In [ ]:
data_q_spot.groupby(["year", "qtr", "flg_isWeekend"])[q_cols].mean()

# 3. EDA

In [ ]:
data[["spot_price", "GP48_total"]].describe()

In [ ]:
data.groupby(["year"])[["spot_price"]].describe()

In [ ]:
weekdays = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
weekday_map = dict(zip(range(0,7+1), weekdays))
quartiles_map = dict(zip(range(0,4+1), ["1st_quartile", "2nd_quartile", "3rd_quartile", "4th_quartile"]))

In [ ]:
features_eda = ["spot_price_mave52", "spot_price_mave7", "spot_price_diff1", "spot_price_diff7", "spot_price_lag1", "spot_price_lag7"]
data_eda = data.copy()

data_eda["spot_price_mave52"] = data_eda["spot_price"].rolling(52).mean()
data_eda["spot_price_mave7"] = data_eda["spot_price"].rolling(7).mean()
data_eda["spot_price_std52"] = data_eda["spot_price"].rolling(52).std()

data_eda["spot_price_diff1"] = data_eda["spot_price"].diff(1)
data_eda["spot_price_diff7"] = data_eda["spot_price"].diff(1)

data_eda["spot_price_lag1"] = data_eda["spot_price"].shift(1)
data_eda["spot_price_lag7"] = data_eda["spot_price"].shift(7)

data_eda["spot_price_t1"] = data_eda["spot_price"].shift(-1)

data_eda["weekday"] = data_eda["weekday"].replace(weekday_map)
data_eda["spot_price_q"] = data_q_spot["spot_price_q"].replace(quartiles_map)

for col in features_gen_pct:
    data_eda[col] = data_eda[col]*100

data_eda = data_eda.dropna()

## 3.1 Time series analysis

### 3.1.1. Time series visualization

In [ ]:
fig, axs = plt.subplots(1,2, sharey=True)
data_eda.plot(y=["spot_price", "spot_price_mave52"], ax=axs[0])
plt.suptitle("Spot price time series visualization")
axs[0].set_ylabel("€/MWh")

sns.distplot(data_eda["spot_price"], vertical=True, ax=axs[1])
plt.show()

### 3.1.2. Seasonality analysis

In [ ]:
fig, axs = plt.subplots(1,2, sharey=True, figsize=(22,8))
sns.boxplot(x="qtr", y="spot_price", data=data_eda, ax=axs[0])
axs[0].set_title("Quarterly distribution")
axs[0].set_ylabel("€/MWh")
sns.boxplot(x="weekday", y="spot_price", data=data_eda, ax=axs[1], order=weekdays)
axs[1].set_title("Weekday distribution")
axs[1].set_ylabel("")
plt.suptitle("Spot price  seasonality analysis")
plt.show()

There are two clear seasonal patterns, one long term pattern indicating that in quarter 3 and 4 prices tend to go up, and a noticable weekly scheme pointing out that in weekends, prices decrease (on average)

Rather than performing a traditional time series analysis, let's figure out what variables are driving this seasonal differences. Is demand (measured by energy generated) behind these differences?

In [ ]:
fig, axs = plt.subplots(1,2, sharey=True, figsize=(22,8))
sns.boxplot(x="qtr", y="GP48_total", data=data_eda, ax=axs[0])
axs[0].set_title("Quarterly distribution")
sns.boxplot(x="weekday", y="GP48_total", data=data_eda, ax=axs[1], order=weekdays)
axs[1].set_title("Weekly distribution")
plt.suptitle("Daily average generation (MWh) seasonality analysis")
plt.show()

Weekly seasonality is explained by a decrease in electricity demand, but there is not a clear relationship yerly level.

There are some techonologies with OPEX and very high CAPEX, like wind and hydro utilities, that tend to bid at low price because the underlying resource is intermitent. Let's check this hypothesis visually

In [ ]:
fig, axs = plt.subplots(1,2, sharey=True, figsize=(22,8))
sns.boxplot(x="qtr", y="GP48_wind_pct", data=data_eda, ax=axs[0])
axs[0].set_ylabel("%")
axs[0].set_title("Wind generation")
sns.boxplot(x="qtr", y="GP48_hydro_total_pct", data=data_eda, ax=axs[1])
plt.suptitle("Daily average % generation quaterly distributions")
axs[1].set_ylabel("%")
axs[1].set_title("Hidro generation")

plt.show()

As the previous plot states, spot price in quarter 3 and four is historically higher, this is mainly due an important reduction in wind and hidro generated energy. In fact, quater 1 price is almost the lowest, and demand is the highest, however, an outstading wind generation drives downs prices

In [ ]:
fig, axs = plt.subplots(1,2, sharey=True, figsize=(22,8))
sns.barplot(x="mon", y="GP48_nuclear", data=data_eda, color="blue", saturation =0.25, ax=axs[0])
axs[0].set_title("Nuclear")
axs[0].set_ylabel("MWh")
sns.barplot(x="mon", y="GP48_CC", data=data_eda, color="blue", saturation =0.25, ax=axs[1])
axs[1].set_title("Natural gas combined cycle")
axs[1].set_ylabel("")
plt.suptitle("Average daily P48 generation by month")

plt.show()

In addition, in quarters 3 and 4 there is a decrease in nuclear generation, tipically this kind of utillities (second generation Pressurized Water Reactor or PWR) halts their production twice a year to refill nuclear fuel (notice months 5 and 11). Especially, in quarter 4, this overlaps with wind and hydro generation, leading to a more expensive technologies increased generation share, like combined Cycles

### 3.1.3. Trend analysis

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(data=data_eda[:"2018"], x="year", y="spot_price")
plt.title("Spot price distribution over years")
plt.ylabel("€/MWh")
plt.xticks(rotation=45)
plt.show()

There is a sustained increase en yearly mean daily spot price, tendency that is only broken at 2016. Let's study this phenomenon in detail

In [ ]:
agg_price = data_eda[:"2018"].groupby(["year"])["spot_price"].mean()
agg_gen = data_eda[:"2018"].groupby(["year"])["GP48_total", "GP48_wind", "GP48_hydro_total", "GP48_CC"].sum()

agg_gen["GP48_wind_hydro_year_pct"] = agg_gen[["GP48_wind", "GP48_hydro_total"]].sum(axis=1).divide(agg_gen["GP48_total"], axis=0)*100
agg_gen["GP48_CC_pct"] = agg_gen[["GP48_CC"]].sum(axis=1).divide(agg_gen["GP48_total"], axis=0)*100


In [ ]:
fig, axs = plt.subplots(1,5, sharey=True, figsize=(36,8))

plt.suptitle("Yearly spot price analysis")
sns.barplot(data=data_eda[:"2018"], x="spot_price", y="year", orient ="h", ax=axs[0])
axs[0].set_title("spot price")
axs[0].set_xlabel("€/MWh")

agg_gen.plot(y=["GP48_wind_hydro_year_pct", "GP48_CC_pct"], kind="barh", ax=axs[1])
axs[1].set_title("Wind+Hydro and CC")
axs[1].set_xlabel("%")

sns.barplot(data=data_eda, x='EUA_CO2', y="year", orient ="h",ax=axs[2])
axs[2].set_title("Average EUA CO2 price")
axs[2].set_xlabel("€/ton CO2")

sns.barplot(data=data_eda, x='coal_API2', y="year", orient ="h",ax=axs[3])
axs[3].set_title("Average coal API2 price")
axs[3].set_xlabel("USD/ton")

sns.barplot(data=data_eda, x='NG_TTF', y="year", orient ="h",ax=axs[4])
axs[4].set_title("Average NG TTF price")
axs[4].set_xlabel("€/MWh")

plt.show()

This array of plots points out the following insights:
* 2015 spot price was quite high compared with 2014 and 2016, due to a lower hydro+wind generation and quite high NG prices
* The cheapest year of the series is 2016, due to de fact that coal and NG prices went wodn, as well a as a strong hydro+wind generation
* In 2017 there is a remarkable increase in coal and NG prices, at the same time, a very poor hdyro+wind generation. These circumstances trigger a high spot price
* Fossil fuels upward trend continues in 2018 and a sharp increase in EUA CO2 prices also takes place. These two effects cannot be counterbalanced by a recovery in hydro+wind generation

Let's quantify the relationship between spot price and commidities price data (fossil fuel prices and EUA CO2 prices)


In [ ]:
agg_qtr_fin = data_eda.groupby(["qtr_idx"])[["EUA_CO2", "coal_API2", "NG_TTF", "spot_price"]].mean()
agg_year_fin = data_eda.groupby(["year"])[["EUA_CO2", "coal_API2", "NG_TTF", "spot_price"]].mean().apply(np.log)

In [ ]:
agg_qtr_fin.head() 

In [ ]:
fig, axs = plt.subplots(1,3, sharey=False, sharex=False, figsize=(18,8))

plt.suptitle("Spot price long term relationship with commodities price: Quarterly averages")
sns.regplot(data=agg_qtr_fin, x="EUA_CO2", y="spot_price", ax=axs[0])
axs[0].set_ylim(0,75)
axs[0].set_ylabel("spot price (€/MWh)")
axs[0].set_xlabel("EUA CO2 (€/ton)")
sns.regplot(data=agg_qtr_fin, x="coal_API2", y="spot_price", ax=axs[1])
axs[1].set_ylim(0,75)
axs[1].set_ylabel("")
axs[1].set_xlabel("Coal API2 (USD/ton)")
sns.regplot(data=agg_qtr_fin, x="NG_TTF", y="spot_price", ax=axs[2])
axs[2].set_xlim(10,30)
axs[2].set_ylim(0,75)
axs[2].set_ylabel("")
axs[2].set_xlabel("Natural Gas TTF (€/MWh)")

for idx, col in enumerate(["EUA_CO2", "coal_API2", "NG_TTF"]):
    corner_idx=agg_qtr_fin[col].min()
    slope, intercept, r_value, p_value, std_err = linreg(x=agg_qtr_fin[col],y=agg_qtr_fin["spot_price"])
    for msg_idx, msg in enumerate(['$y={:.2f}+{:.2f}x$'.format(intercept, slope), '$R^2={:.2f}$'.format(r_value)]):
        axs[idx].text(corner_idx,(msg_idx+1)*5, msg, style='italic', ha="left")
plt.show()

This confirms that these commodities are strong spot prices drivers in long term, as regression lines are quite well fitted at quarterly aggregated data

### 3.2. Correlation Analysis

Pearson correlation indicates if two variables are linearly related. Explained visually: it is close to 1 in absolute terms if data fit well to a line, and correlation sign depends on this adjusted line slope. It is important to emphasize that other relationships rather than linear, are not properly captured by this score (try Spearman correlation).

This correlation analysis is carried out in two subset of features:
* Autocorrelation analysis
* Rolling stats: It may capture past influence in future values, as well as it may model seasonality
* Generation share by technology: It will indicate which technologies are the main price drivers
* Commodity price: As seen before, there is a relationship when data is aggregated quaterly, let's check if it might model trend pattern

### 3.2.1 Autocorrelation analysis

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

fig, axs = plt.subplots(1,2, sharey=True)
plot_acf(data_eda["spot_price"],lags=30, ax=axs[0])
axs[0].set_title("Raw time series autocorrelation")
plot_acf(data_eda["spot_price_diff1"],lags=30, ax=axs[1])
axs[1].set_title("Differenced time series autocorrelation")
plt.show()

Taking first differences (first order), it results that Autocorrelation Plot that shows lagged values may carry useful information. ARIMA model lovers will expect a more in depth discussion, but for a ML forecasting model it is enough

#### 3.2.2 Rolling stats: Spot price

In [ ]:
eda_corr2 = data_eda[["spot_price_t1"] + features_eda].corr()
sns.heatmap(eda_corr2, annot=True, cmap="YlGnBu")
plt.show()

In [ ]:
l =  ["spot_price_lag1","spot_price_lag7","spot_price_mave7","spot_price_mave52"]
fig, axs = plt.subplots(2,len(l), figsize=(32, 18))
for idx, col in enumerate(l):
    sns.distplot(data_eda[col], ax=axs[0,idx])
    sns.regplot(data=data_eda, x=col, y="spot_price_t1",ax=axs[1,idx])
plt.suptitle("Correlation with generation share by main techology")
plt.show()

As seen in Autocorrelation plots, there is a lot of information in lagged values, as well as in moving averages

In [ ]:
data_eda[l + ["spot_price_t1"]].corr().loc[l]

#### 3.2.3. Generation share ratio by technology

In [ ]:
eda_corr1 = data_eda[["spot_price_t1"] + features_gen_pct].corr()
sns.heatmap(eda_corr1, annot=True, cmap="YlGnBu")
plt.show()

In [ ]:
l =  ["GP48_coal_pct","GP48_CC_pct","GP48_wind_pct","GP48_hydro_total_pct"]
fig, axs = plt.subplots(2,len(l), figsize=(32, 18))
for idx, col in enumerate(l):
    sns.distplot(data_eda[col], ax=axs[0,idx])
    sns.regplot(data=data_eda, x=col, y="spot_price_t1",ax=axs[1,idx])
plt.suptitle("Correlation with generation share by main techology")
plt.show()

In [ ]:
data_eda[l + ["spot_price_t1"]].corr().loc[l]

* It is an interesting fact, that combined cycles and coal utilities yield the highest correlation scores, and an upward fit line slope. It can be seen, that the relationship is not linear, let's apply a transformation to x variable (log)
* On the other hand, wind and hydro also impact spot price but in the opposite way, as we previously explained, cost structure and resource intermitence are behind this behavior. In addition, the relationship is linear.



To ilustrate the fact that not every relationship must be linear, let's fit a log function to combined cycles data

In [ ]:
plt.figure(figsize=(8,6))
sns.regplot(data=data_eda, x="GP48_CC_pct", y="spot_price_t1",logx = True, fit_reg =True)
plt.ylabel("Spot price (€/MWh)")
plt.xlabel("Combined Cycle % generation")
plt.title("Log fitted curve: spot_price=f(CC)")

plt.show()

In [ ]:
l =  ["EUA_CO2", "NG_TTF", "coal_API2"]
fig, axs = plt.subplots(2,len(l), sharex=False, figsize=(32, 18))
for idx, col in enumerate(l):
    sns.distplot(data_eda[col], ax=axs[0,idx])
    sns.regplot(data=data_eda, x=col, y="spot_price_t1",ax=axs[1,idx])
plt.suptitle("Correlation with generation share by main techology")
plt.show()

In [ ]:
data_eda[l + ["spot_price_t1"]].corr().loc[l]

As we could see in the last section, this variables influence at quarterly level is quite high, and the correlation score maintains at dailiy level, although data is very scattered around regresion line, due to seasonal (weekly and yearly patterns) and variability due to trend. A complex enough algorithm must device a way to combine seasonal and trend features in order to deal with this time structure

# 4. Feature Exploration and Engineering

The goal of this section is to handcraft some interesting features that will significantly improve our model performance. Moreover, some automatic feature engineering will be performed to exploit EDA insigths. 

## 4.1. Wind generation

In [ ]:
data_points_year= '2017'
data_pints_order=["1st_quartile", "2nd_quartile", "3rd_quartile", "4th_quartile"]

sns.barplot(x="qtr", y="GP48_wind_pct", data=data_eda, ci="sd", fill=False, edgecolor='purple', linewidth=10, alpha=.5)
sns.swarmplot(x="qtr", y="GP48_wind_pct", hue="spot_price_q", data=data_eda[data_points_year], hue_order =data_pints_order, alpha=.5, s=12, dodge=True)
plt.text(0.25,50,"Data points year: {}".format(data_points_year))

plt.ylabel("Average wind generation %")
plt.xlabel("Quarter")
plt.title("Quarterly spot price distribution vs wind generation: Data points year 2017")
plt.show()

This bar displays in bars how much generation comes from wind by quarter, computed taking averages from daily percentages. At each quarter, it is overlayed a distribution of scattered points. Those points height is daily wind generation percentage and colour represents the spot price quartile that each one belongs to. Quartiles are computed by quater and weekday (as wee saw that spot price is strongly influenced by these variables) as well as by year, because there are significant differences from one to another. Due to 

It can be clearly seen that higher points are blue and yellow, therefore they were "cheap" days that belong to lower quartiles (1st and 2nd), whereas low points are green and red, so they were expensive days when wind generation was low.

This kind of interaction will be frequent and our Machine Learning algorithm must be complex enough to model them

## 4.2. Exports and Imports

In [ ]:
l = ["GP48_export_pct", "GP48_export","GP48_import_pct", "GP48_import"]
data_eda[l +  ["spot_price_t1"]].corr().loc[["spot_price_t1"], l]

In [ ]:
data_eda["GP48_export_mmax"] = data_eda["GP48_export"].rolling(120).min()
data_eda["GP48_import_mmax"] = data_eda["GP48_import"].rolling(120).max()
data_eda["GP48_export_pct_mave"] = data_eda["GP48_export_pct"].rolling(180).mean()
data_eda["GP48_import_pct_mave"] = data_eda["GP48_import_pct"].rolling(180).mean()

data_eda["GP48_export_pct_over_mmax"] = data_eda["GP48_export"].div(data_eda["GP48_export_mmax"])
data_eda["GP48_import_pct_over_mmax"] = data_eda["GP48_import"].div(data_eda["GP48_import_mmax"])
data_eda[["spot_price_t1", "GP48_export_pct_over_mmax","GP48_export_mmax", "GP48_export_pct_mave",
          "GP48_import_pct_over_mmax", "GP48_import_mmax", "GP48_import_pct_mave"]].corr()

In [ ]:
data_eda[[ "GP48_export_pct","GP48_export_pct_mave"]].corr()

In [ ]:
features_eng =  ["GP48_export_pct_mave180"]
data["GP48_export_pct_mave180"] = data["GP48_export_pct"].rolling(180).mean()

As raw data, exports and imports are not good predictors, however, trying to compute this figure as percentage over rolling maximum does not yield better results. However exports rolling 120-days max seems to be the best engineering feature, especially at extreme values. This makes senses, since the higher the price, the lower chances to export energy to other markets linke French one, where prices are usually lower.

## 4.3. Pumping: Turbination and Consumption

In [ ]:
l = ["GP48_pumping_con_pct", "GP48_pumping_con","GP48_pumping_tur_pct", "GP48_pumping_tur"]
data_eda[l +  ["spot_price_t1"]].corr().loc[["spot_price_t1"], l]

In [ ]:
data_eda["GP48_pumping_con_mmax"] = data_eda["GP48_pumping_con"].rolling(240).min()
data_eda["GP48_pumping_tur_mmax"] = data_eda["GP48_pumping_tur"].rolling(240).max()
data_eda["GP48_pumping_con_pct_mave"] = data_eda["GP48_pumping_con_pct"].rolling(180).mean()
data_eda["GP48_pumping_tur_pct_mave"] = data_eda["GP48_pumping_tur_pct"].rolling(7).mean()


data_eda["GP48_pumping_con_pct_over_mmax"] = data_eda["GP48_pumping_con"].div(data_eda["GP48_pumping_con_mmax"])
data_eda["GP48_pumping_tur_pct_over_mmax"] = data_eda["GP48_pumping_tur"].div(data_eda["GP48_pumping_tur_mmax"])

data_eda[["GP48_pumping_con_pct_over_mmax", "GP48_pumping_con_mmax", "GP48_pumping_con_pct_mave",
          "GP48_pumping_tur_pct_over_mmax", "GP48_pumping_tur_mmax", "GP48_pumping_tur_pct_mave", 
          "spot_price_t1"]].corr()[["spot_price_t1"]].T

In [ ]:
data_eda[["GP48_pumping_con_pct","GP48_pumping_tur_pct", "GP48_pumping_con_pct_mave","GP48_pumping_tur_pct_mave",
         "GP48_pumping_con_pct_over_mmax", "GP48_pumping_tur_pct_over_mmax"]].corr()

In [ ]:
features_eng = features_eng + ["GP48_pumping_con_pct_mave180", "GP48_pumping_tur_pct_mave7","GP48_pumping_tur_pct_over_mmax240"]

data["GP48_pumping_tur_pct_over_mmax240"] = data["GP48_pumping_tur"].div(data["GP48_pumping_tur"].rolling(240).max())
data["GP48_pumping_con_pct_mave180"] = data["GP48_pumping_con_pct"].rolling(180).mean()
data["GP48_pumping_tur_pct_mave7"] = data["GP48_pumping_tur_pct"].rolling(7).mean()

Turbining and consuming energy as raw predictors yield a good correlation, but consuming improves significantly if a percentage over max and a moving average are computed (moreover this features are not very correlated). Regarding turbined water, the best feature is to calculate a moving average (correlation almost doubles)

## 4.4. Nuclear power

In [ ]:
l = ["GP48_nuclear", "GP48_nuclear_pct"]
data_eda[l +  ["spot_price_t1"]].corr().loc[["spot_price_t1"], l]

In [ ]:
data_eda["GP48_nuclear_mmax"] = data_eda["GP48_nuclear"].rolling(180).max()
data_eda["GP48_nuclear_mave"] = data_eda["GP48_nuclear"].rolling(180).mean()
data_eda["GP48_nuclear_pct_mmax"] = data_eda["GP48_nuclear_pct"].rolling(180).max()
data_eda["GP48_nuclear_pct_mave"] = data_eda["GP48_nuclear_pct"].rolling(7).mean()

data_eda["GP48_nuclear_pct_over_mmax"] = data_eda["GP48_nuclear"].div(data_eda["GP48_nuclear_mmax"])

data_eda[["spot_price_t1", "GP48_nuclear_pct_over_mmax","GP48_nuclear","GP48_nuclear_pct", "GP48_nuclear_mmax",
          "GP48_nuclear_pct_mmax", "GP48_nuclear_mave", "GP48_nuclear_pct_mave"]].corr()

In [ ]:
features_eng = features_eng + ["GP48_nuclear_pct_mmax180"]

data["GP48_nuclear_pct_mmax180"] = data["GP48_nuclear_pct"].rolling(180).max()

Additionally to compute dayly nuclear generation share (GP48_nuclear_mave) taking rolling max also yield a good predictor (GP48_nuclear_pct_mmax), as well as a cmputing rolling average on generated nuclear energy. Moreover, correlation among these features is not very high

## 4.5. Solar Energy

In [ ]:
l = ["GP48_PV", "GP48_PV_pct", "GP48_thermosolar", "GP48_thermosolar_pct"]
data_eda[l +  ["spot_price_t1"]].corr().loc[["spot_price_t1"], l]

In [ ]:
data_eda[["GP48_PV", "GP48_thermosolar"]].plot(figsize=(8,6))
plt.show()

In [ ]:
data_eda["GP48_solar"] = data_eda[["GP48_PV", "GP48_thermosolar"]].sum(axis=1)
data_eda["GP48_solar_pct"] = data_eda["GP48_solar"].div(data_eda["GP48_total"])*100
data_eda[["spot_price_t1", "GP48_PV_pct", "GP48_thermosolar_pct", "GP48_solar", "GP48_solar_pct"]].corr()

In [ ]:
data_eda["GP48_PV_pct_mave"] = data_eda["GP48_PV_pct"].rolling(7).mean()
data_eda["GP48_solar_pct_mave"] = data_eda["GP48_solar_pct"].rolling(7).mean()

data_eda[["spot_price_t1", "GP48_PV_pct", "GP48_thermosolar_pct", "GP48_solar", "GP48_solar_pct", "GP48_PV_pct_mave", "GP48_solar_pct_mave"]].corr()

Sun energy effect on Spanish market price is not very significant, this may be due to a very low installed capacity. During year 2019-2020 it is expected that this espectaculary changes, therefore marginal depresssing effect on market price may be very noticably.

My two cents on the issue:

* PV energy was heavily subsidized via Feed in Tariffs (FIT) that, at it greatest peak, a premium of 450 €/MWh was given. Nowadays, this effort is not paying off.
* On the other hand, a strong expansion of this kind of energy (withouth subsidies) is expected and it may radically change Spanish electricity market. 

## 4.6. Coal and Natural Gas

In [ ]:
data_eda["GP48_CC_pct_x_TTF"] = data_eda["NG_TTF"]*data_eda["GP48_CC"]
data_eda["GP48_coal_pct_x_API2"] = data_eda["GP48_coal"]*data_eda["coal_API2"]
data_eda["GP48_coal_CC_pct_x_CO2"] = (data_eda["GP48_coal"]+data_eda["GP48_CC"])*data_eda["EUA_CO2"]

data_eda[["NG_TTF","GP48_CC_pct", "GP48_coal_pct", "coal_API2", "GP48_CC_pct_x_TTF", "GP48_coal_pct_x_API2", "GP48_coal_CC_pct_x_CO2", "spot_price_t1"]].corr()

In [ ]:
data_eda["NG_TTF_mave"] = data_eda["NG_TTF"].rolling(30).mean()
data_eda["coal_API2_mave"] = data_eda["coal_API2"].rolling(30).mean()
data_eda["EUA_CO2_mave"] = data_eda["EUA_CO2"].rolling(30).mean()
                                          
data_eda[["spot_price_t1", "NG_TTF","NG_TTF_mave", "coal_API2", "coal_API2_mave", "EUA_CO2", "EUA_CO2_mave"]].corr()

In [ ]:
features_eng = features_eng + ["NG_TTF_mave30", "coal_API2_mave30", "EUA_CO2_mave30"]

data["NG_TTF_mave30"] = data["NG_TTF"].rolling(30).mean()
data["coal_API2_mave30"] = data["coal_API2"].rolling(30).mean()
data["EUA_CO2_mave30"] = data["EUA_CO2"].rolling(30).mean()

This kind of interaction yields a better correlation score, but also a higher multicollinearity, as cross features are highly correlated among themselves. In this situation, it is better to let the machine learning algorithm to figure out interactions. It is interesting to point out that this happens because commodities prices also affects to % generation

In [ ]:
features_eng

## 4.6 Feature standardization

In [ ]:
std_compute = ["spot_price", "GP48_total"]
features_std =[x + "_std" for x in std_compute]

data_raw = data[std_compute].copy()
scaler = StandardScaler().fit(data_raw)
new_cols = scaler.transform(data_raw)

for i in range(0, new_cols.shape[1]):
    data[features_std[i]] = new_cols[:,i]

data[std_compute + features_std].head()

Features are stantardize so that scale does not impact on Machine Learning algorithms performace. This can be an issue with Neural Nets, Ridge and Lasso Regression, as well as SVM or KNN algorithms. Let's check that this transformation does not change feature distribution

In [ ]:
fig, axs = plt.subplots(2,2)
plt.suptitle("Standardization effects")
for idx, col in enumerate(std_compute):
    sns.distplot(data[col], kde=True, ax=axs[0,idx])
    
for idx, col in enumerate(features_std):
    sns.distplot(data[col],  kde=True, ax=axs[1,idx])

## 4.7. Rolling features

In [ ]:
#Forecasting horizon
forecating_hor = 30
features_roll = []
targets = []

#Compute targets:
for t in range(1,forecating_hor+1):
    data["target_spot_price_t{}".format(t)] = data["spot_price"].shift(-t)
    targets.append("target_spot_price_t{}".format(t))

for feat in features_std:
    for t in [1,3,7,14]:
        data[feat + "_lag{}".format(t)] = data[feat].shift(t)
        features_roll.append(feat + "_lag{}".format(t))

#only spot price
for t in [7, 30]:
    data[["spot_price_std_mave{}".format(t), 
          "spot_price_std_mstd{}".format(t),
          "spot_price_std_mmax{}".format(t),
          "spot_price_std_mmin{}".format(t)]]\
         = data["spot_price_std"].rolling(t).agg([np.mean, np.std, np.max, np.min])
    
    features_roll = features_roll + ["spot_price_std_mave{}".format(t), 
          "spot_price_std_mstd{}".format(t),
          "spot_price_std_mmax{}".format(t),
          "spot_price_std_mmin{}".format(t)]
    
    for lag in [365]:
        data["spot_price_std_mave{}_lag{}".format(t, lag)] =\
            data["spot_price_std_mave{}".format(t)].shift(lag)
        
        features_roll.append("spot_price_std_mave{}_lag{}".format(t, lag))

In [ ]:
features_full = features_time + features_gen_pct + features_eng + features_std + features_roll
features = ["feat_" + x for x in features_full]
columns =  ["year", "qtr", "mon","weekday", "flg_isWeekend"] + targets + features

feat_rename_dict = dict(zip(features_full,features))

data_fte = data.rename(columns=feat_rename_dict).dropna()[columns]

In [ ]:
corr_df = data_fte[targets[0:1] + features].corr().T[targets[0:1]].loc[features]

In [ ]:
k_features = 10

fig, axs = plt.subplots(1,3)

corr_df_srt = corr_df.sort_values(by=["target_spot_price_t1"], ascending=False)
top_pos_corr = corr_df_srt[corr_df_srt["target_spot_price_t1"]>0].head(k_features)
top_neg_corr = corr_df_srt[corr_df_srt["target_spot_price_t1"]<0].tail(k_features)
                                                       
top_pos_corr.sort_values(by="target_spot_price_t1").plot.barh(ax=axs[0])
top_neg_corr.plot.barh(color="lightcoral", ax=axs[2])

axs[0].set_xlim(0, 1) 
axs[2].set_xlim(0, -1.)

for ax in axs:
    ax.set_xlabel(r"Pearson correlation $\rho$")
fig.delaxes(axs[1])

plt.suptitle("Top {} features most linearly correalted to target variable".format(k_features))
plt.show()

In [ ]:
data_fte.info()

In [ ]:
data_fte.to_csv(PATH + outfile, index_label='date')